In [1]:
from collections import defaultdict

with open('inputs/15.txt') as file:
    input = file.read()


In [2]:
TRACE = 0
def run(mem):
    i = 0
    rel = 0
    def get(j,m):
        p = i+j
        if m == 0:
            p = mem[p]
        elif m == 2:
            p = mem[p] + rel
        return p
    while 1:
        s = ('0000'+str(mem[i]))[-5:]
        a,b,c,op = int(s[0]), int(s[1]), int(s[2]), int(s[3:])
        if op == 1:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]}+{mem[y]}={mem[x] + mem[y]} @ {p}')
            mem[p] = mem[x] + mem[y]
            i += 4
        elif op == 2:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]}*{mem[y]}={mem[x] * mem[y]} @ {p}')
            mem[p] = mem[x] * mem[y]
            i += 4
        elif op == 3:
            p = get(1,c)
            mem[p] = yield
            if TRACE: print(f'{i}: input {mem[p]} @ {p}')
            i += 2
        elif op == 4:
            p = get(1,c)
            if TRACE: print(f'{i}: out {mem[p]} @ {p}')
            yield mem[p]
            i += 2
        elif op == 5:
            x = get(1,c)
            y = get(2,b)
            if TRACE: print(f'{i}: if {mem[x]} jump {mem[y]}')
            if mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 6:
            x = get(1,c)
            y = get(2,b)
            if TRACE: print(f'{i}: if not {mem[x]} jump {mem[y]}')
            if not mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 7:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]} < {mem[y]} => {p}')
            mem[p] = int(mem[x] < mem[y])
            i += 4
        elif op == 8:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]} == {mem[y]} => {p}')
            mem[p] = int(mem[x] == mem[y])
            i += 4
        elif op == 9:
            x = get(1,c)
            if TRACE: print(f'{i}: rel = {rel} + {mem[x]}')
            rel += mem[x]
            i += 2
        elif op == 99:
            return
        else:
            raise Exception('Unknown opcode', op)
            break


In [3]:
def walk(turn,f):
    prog = list(map(int,input.split(',')))
    proc = run(prog)
    next(proc)
    p,d = 0,0
    dc = [1,4,2,3]
    dp = [-1j,1,1j,-1]

    while 1:
        np = p + dp[d]
        res = proc.send(dc[d])
        f[np] = res
        if res == 0:
            d = (d+turn+4)%4
        elif res == 1:
            p = np
            d = (d+turn+6)%4
        elif res == 2:
            p = np
            break

    return f

def tlbr(f):
    t,l,b,r = 0,0,0,0
    for p in f:
        t = min(t,int(p.imag))
        l = min(l,int(p.real))
        b = max(b,int(p.imag))
        r = max(r,int(p.real))
    return t,l,b,r

def shortest(f):
    nxt = [0]
    dist = defaultdict(lambda:1000)
    dp = [-1j,1,1j,-1]
    for step in range(1000):
        if len(nxt)==0: break
        cur = nxt
        nxt = []
        for p in cur:
            if dist[p]<=step: continue
            if f[p]==2:
                backtrack(f,dist,p)
                return step
            dist[p] = step
            for d in dp:
                if f[p+d]!=0: nxt.append(p+d)

def backtrack(f,dist,p):
    dp = [-1j,1,1j,-1]
    while dist[p]!=0:
        if f[p]==1: f[p] = 5
        for d in dp:
            if dist[p+d]<dist[p]:
                p = p+d
                break

def pretty(f):
    t,l,b,r = tlbr(f)
    print('')

    sym = '⬜⚫⭐⬛♿⭕⛔'
    for y in range(t,b+1):
        row = ''
        for x in range(l,r+1):
            i = 4 if x==0 and y==0 else f[complex(x,y)]
            row += sym[i]
        print(row)


In [4]:
%%time

f = defaultdict(lambda:-1)
walk(1,f)
walk(-1,f)
print(shortest(f))
pretty(f)


220

⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔
⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜
⛔⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⛔
⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜
⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜
⬜⚫⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⛔⬜⬜⚫⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜
⛔⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜
⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜
⛔⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⛔⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⛔
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜
⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⛔
⬜⚫⚫⚫⬜⚫⚫⚫⬜⭕⭕⭕⬜⚫⬜⚫⬜⚫⚫⚫⬜♿⭕⭕⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⬜⭕⭕⭕⬜
⛔⬜⬜⬜⬜⚫⬜⬜⬜⭕⬜⭕⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⭕⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⭕⬜⭕⬜
⬜⚫⚫⚫⚫⚫⬜⭕⭕⭕⬜⭕⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⭕⬜⚫⚫⚫⬜

In [5]:
def oxygenate(f):
    filled = defaultdict(lambda:-1)
    tofill = 0
    for p in f:
        if f[p]==1 or f[p]==5:
            filled[p] = 0
            tofill += 1
        elif f[p]==2:
            start = p
        
    nxt = [start]
    dp = [-1j,1,1j,-1]
    for step in range(1,1000):
        if len(nxt)==0: break
        cur = nxt
        nxt = []
        for p in cur:
            for d in dp:
                if filled[p+d]!=0: continue
                filled[p+d] = 1
                tofill -= 1
                if tofill == 0:
                    return step
                nxt.append(p+d)
    

In [6]:
%%time

oxygenate(f)


Wall time: 1.99 ms


334